In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import glm_model as gm
from glm_utils import *
import tensorflow as tf

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#finally get rid of the ugly ass allen code. We import the pickled design 
#matrix stimulus, which is PCA projection of the images, and the downsampled
#traces. 

import pickle

pkl_file = open('./boc/formatted/LMcelldata')
data = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/LMstimdata')
stim = pickle.load(pkl_file)
pkl_file.close()

In [3]:
#get the number of features, and the number of training examples. 
l, n =  stim.shape

In [4]:
from statsmodels.tsa.tsatools import lagmat
from sklearn.cross_validation import train_test_split
from scipy.stats import linregress
from itertools import product
import pyprind
import sys

n_pcs = range(10, 110, 20)
non_lin = [3]
non_lin_dict = {1: "sigmoid", 2: "relu", 3: "exponential"}
alpha = 0

#empty dictionary of the best model we've fit for each cell
best_model = {k: (-np.inf, None) for k in data.keys()}

#make a dictionary of dictionaries to store the results
scores_dict = {k: {} for k in data.keys()}

bar = pyprind.ProgBar(len(n_pcs)*len(non_lin))

for (l, n) in product(non_lin, n_pcs):
    print "testing non-linearity: ", non_lin_dict[l], "... testing ", n, " pc's..."
    sys.stdout.flush()
    
    #lagged_stim = lagmat(stim[:, 0:n], maxlag = nlag, trim = None)[1:-1]         
    bar.update()
    
    for c in data.keys():
        X_train, X_test, y_train, y_test = train_test_split(stim[:, 0:n], np.array(data[c]))     
        weight_init = np.linalg.pinv(X_train).dot(y_train)[:, np.newaxis]

        if l == 1:
            nonlin = tf.nn.sigmoid
        if l == 2:
            nonlin = tf.nn.relu
        if l == 3: 
            nonlin = tf.exp
        
        model = gm.gaussian_GLM(weight_init, non_lin = nonlin, verbose = False, train_params = False, bias_init = 0, scale_init = 1)
        cv, loss = model.fit(X_train, y_train[:, np.newaxis], X_test, y_test[:, np.newaxis], max_iters = 150) 
        w, o, s  = model.get_params()
    
        _, _, r2_test,_, _ = linregress(np.squeeze(model.predict_trace(X_test)), np.squeeze(y_test))
        _, _, r2_train,_, _ = linregress(np.squeeze(model.predict_trace(X_train)), np.squeeze(y_train))

        scores_dict[c][(l, n)] = (r2_test, r2_train) 

        if r2_test > best_model[c][0]:
            best_model[c] = (r2_test, (l, n))
            print "new best model for cell ", c, ". r2 test: ", r2_test


0%  100%
[     ]

testing non-linearity:  exponential ... testing  10  pc's...


[#    ] | ETA: 00:00:00

testing non-linearity:  exponential ... testing  30  pc's...


[##   ] | ETA: 00:01:07

...

[###  ] | ETA: 00:01:32

testing non-linearity:  exponential ... testing  70  pc's...


[#### ] | ETA: 00:01:10

testing non-linearity:  exponential ... testing  90  pc's...


[#####] | ETA: 00:00:00
Total time elapsed: 00:07:22
